In [ ]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt 
from datetime import datetime

from scipy.stats.mstats import winsorize

import os

import warnings
warnings.filterwarnings("ignore")

# Preprocessing

In [ ]:
### COMORBIDITIES ###

CMB = pd.read_excel()
CMB.sort_values(by=['Dyad','Date'], inplace=True)
CMB.drop_duplicates(subset=['Dyad', 'Category'], keep='first', inplace=True)

CMB_one_hot_encoded = pd.get_dummies(CMB, columns=['Category'])
CMB_one_hot_encoded.rename(columns=lambda x: x.replace('Category_', ''), inplace=True)

boolean_columns = CMB_one_hot_encoded.select_dtypes(include='bool').columns
CMB_one_hot_encoded[boolean_columns] = CMB_one_hot_encoded[boolean_columns].astype(int)

columns_to_update = CMB_one_hot_encoded.columns.difference(['Dyad', 'Date'])
CMB_one_hot_encoded[columns_to_update] = CMB_one_hot_encoded.groupby('Dyad')[columns_to_update].cummax()

CMB = CMB_one_hot_encoded.copy()

In [ ]:
### DEMOGRAPHICS ###

Dem = pd.read_csv()
Dem.rename(columns={'gender':'Gender', 'year_of_birth_pwd':'YoB', 'education_pwd':'Education',
                    'does_pwd_live_alone':'PLWD Lives Alone'}, inplace=True)

### DIAGNOSIS DATE ###

diagnosis_date = CMB[['Date', 'Dyad', 'Diagnosis']]
diagnosis_date = diagnosis_date[diagnosis_date['Diagnosis']!= 0]
diagnosis_date = diagnosis_date.groupby('Dyad', as_index=False).first()

CMB.drop(columns='Diagnosis', inplace=True)

diagnosis_date.drop(columns='Diagnosis', inplace=True)
diagnosis_date.rename(columns={'Date':'Diagnosis_date'}, inplace=True)

In [ ]:
### MMSE ###

mmse = pd.read_csv()

mmse.dropna(inplace = True) # Drop all rows with any missing data 

mmse_sorted = mmse.sort_values(by=['Dyad', 'date_test_mmse'], ascending=True).sort_index()
mmse_sorted = mmse_sorted.sort_values(by=['Dyad', 'date_test_mmse'], ascending=True)

mmse_sorted['date_test_mmse'] = pd.to_datetime(mmse_sorted['date_test_mmse'])
date_threshold = pd.to_datetime("25/03/2025", format='%d/%m/%Y')
mmse_preapr = mmse_sorted[mmse_sorted['date_test_mmse'] < date_threshold]
mmse = mmse_preapr.copy()

# mmse[mmse.duplicated(subset=['date_test_mmse', 'Dyad'], keep=False)]
# mmse[mmse['Assessment Month']=='unsched']
# mmse[mmse['Assessment Month']=='exit']

# Drop rows with all zero scores
mmse = mmse[mmse['tot_score_mmse']!=0]

# Ex: Drop duplicates
mmse.drop([idx], inplace=True)

# Ex: Update/drop 'exit'
mmse.drop([idx], inplace=True)
mmse.loc[idx, 'Assessment Month'] = 48

In [ ]:
### ADAS-COG ###

adas =  pd.read_csv()

# Removed all rows with notes and no scores (COVID, withdrawals, etc)

# Drop rows with more than 50% missing data and no note
mask = (adas['notes_adascog'].notna()) | (adas.isna().sum(axis=1)<len(adas.columns)/2)
adas = adas[mask]

# For missing values, impute with maximum score 
impute_values = {
    'spoken_language': 5,
    'comprehension_language': 5,
    'word_finding' : 5,
    'remembering_instructions' : 5,
    'word_recall': 10,
    'naming':5,
    'commands': 5,
    'constructional_praxis': 5,
    'delayed_recall' : 10,
    'ideational_praxis' : 5,
    'orientation': 8,
    'word_recognition' :12,
    'maze' :5,
    'number_cancellation': 5
}
for column, impute_value in impute_values.items():
    adas[column].fillna(impute_value, inplace=True)

adas['ADAS-Cog Adjusted Total'] = adas.loc[:, 'spoken_language':'number_cancellation'].sum(axis=1)
adas.drop(columns='total_score_adascog', inplace=True)
adjusted_adas = adas.copy()
uncomplete = adjusted_adas.loc[adjusted_adas['ADAS-Cog Adjusted Total'] > 89]
adjusted_adas_drop_incomp = adjusted_adas[~adjusted_adas['Dyad'].isin(uncomplete['Dyad'])]
adjusted_adas_drop_incomp.drop(['notes_adascog'], axis = 1, inplace = True)

adas_sorted = adjusted_adas_drop_incomp.sort_values(by=['Dyad', 'date_adascog'], ascending=True).sort_index()
adas_sorted = adas_sorted.sort_values(by=['Dyad', 'date_adascog'], ascending=True)
adas_sorted['date_adascog'] = pd.to_datetime(adas_sorted['date_adascog'])
date_threshold = pd.to_datetime("25/03/2025", format='%d/%m/%Y')
adas_preapr = adas_sorted[adas_sorted['date_adascog'] < date_threshold]

adas = adas_preapr.copy()

In [ ]:
### BADL ###

badl = pd.read_csv()

threshold = len(badl.columns) / 2
rows_to_drop = badl[badl.isna().sum(axis=1) > threshold].index # Some missing over 50% due to COVID mix up with assessment packs.
badl.drop(index=rows_to_drop, inplace = True)
badl['Dyad'] = badl['Dyad'].astype(float)

badl_sorted = badl.sort_values(by=['Dyad', 'badl_date'], ascending=True).sort_index()
badl_sorted = badl_sorted.sort_values(by=['Dyad', 'badl_date'], ascending=True)

badl_sorted['badl_date'] = pd.to_datetime(badl_sorted['badl_date'])
date_threshold = pd.to_datetime("25/03/2025", format='%d/%m/%Y')
badl_preapr = badl_sorted[badl_sorted['badl_date'] < date_threshold]
badl_preapr.sort_values(by = ['Dyad', 'badl_date'], ascending=True)
badl=badl_preapr.copy()

badl.sort_values(by = ['Dyad', 'badl_date'], ascending=True, inplace=True)

badl.drop(columns=['notes_badl', 'total_score_badl'], inplace=True)
badl = badl.groupby('Dyad').apply(
    lambda group: group.sort_values('Assessment Month').fillna(method='ffill').fillna(method='bfill')
)

badl = badl.reset_index(drop=True)
badl['BADL Adjusted Total'] = badl.loc[:, 'food_score':'transport_score'].sum(axis=1)
badl.dropna(inplace=True)

# Data prep for modelling

In [ ]:
mmse.rename(columns={'date_test_mmse':'Date', 'tot_score_mmse':'MMSE'}, inplace=True)
mmse = mmse.sort_values(by=['Dyad', 'Assessment Month'])

mmse_12m = mmse.copy()
mmse_12m['MMSE_12m'] = mmse_12m.groupby('Dyad')['MMSE'].shift(-1)
mmse_12m['AM_12m'] = mmse_12m.groupby('Dyad')['Assessment Month'].shift(-1)
mmse_12m['AM_dif'] = mmse_12m['AM_12m']-mmse_12m['Assessment Month']

mmse_12m = mmse_12m[mmse_12m['AM_dif'] == 12]
mmse_12m.drop(columns=['AM_dif', 'AM_12m'], inplace=True)
mmse_12m.dropna(subset=['MMSE_12m'], inplace=True)
mmse_12m.reset_index(drop=True, inplace=True)

mmse_12m.rename(columns=lambda x: x.replace('_mmse', ' (Baseline MMSE)').capitalize(), inplace=True)
mmse_12m.rename(columns=lambda x: x.replace('baseline mmse', 'Baseline MMSE'), inplace=True)
mmse_12m.rename(columns = {'Mmse':'Baseline MMSE', 'Assessment month':'Assessment Month', 'Mmse_12m':'MMSE_12m'}, inplace=True)

In [ ]:
adas.rename(columns={'date_adascog':'Date', 'ADAS-Cog Adjusted Total':'ADAS'}, inplace=True)

adas.rename(columns=lambda x: x.replace('_', ' ').capitalize(), inplace=True)
adas.rename(columns={'Adas':'Baseline ADAS-Cog', 'Assessment month':'Assessment Month'}, inplace=True)
adas.rename(columns = {col: f"{col} (Baseline ADAS)" for col in adas.loc[:, 'Spoken language':'Number cancellation'].columns}, inplace=True)

# Invert ADAS-Cog scores for SHAP interpretability 
adas['Baseline ADAS-Cog'] = 90 - adas['Baseline ADAS-Cog']
adas['Spoken language (Baseline ADAS)'] = 5 - adas['Spoken language (Baseline ADAS)']
adas['Comprehension language (Baseline ADAS)'] = 5 - adas['Comprehension language (Baseline ADAS)']
adas['Word finding (Baseline ADAS)'] = 5 - adas['Word finding (Baseline ADAS)']
adas['Remembering instructions (Baseline ADAS)'] = 5 - adas['Remembering instructions (Baseline ADAS)']
adas['Word recall (Baseline ADAS)'] = 10 - adas['Word recall (Baseline ADAS)']
adas['Naming (Baseline ADAS)'] = 5 - adas['Naming (Baseline ADAS)']
adas['Commands (Baseline ADAS)'] = 5 - adas['Commands (Baseline ADAS)']
adas['Constructional praxis (Baseline ADAS)'] = 5 - adas['Constructional praxis (Baseline ADAS)']
adas['Delayed recall (Baseline ADAS)'] = 10 - adas['Delayed recall (Baseline ADAS)']
adas['Ideational praxis (Baseline ADAS)'] = 5 - adas['Ideational praxis (Baseline ADAS)']
adas['Orientation (Baseline ADAS)'] = 8 - adas['Orientation (Baseline ADAS)']
adas['Word recognition (Baseline ADAS)'] = 12 - adas['Word recognition (Baseline ADAS)']
adas['Maze (Baseline ADAS)'] = 5 - adas['Maze (Baseline ADAS)']
adas['Number cancellation (Baseline ADAS)'] = 5 - adas['Number cancellation (Baseline ADAS)']

adas = adas.sort_values(by=['Dyad', 'Assessment Month'])

In [ ]:
badl.drop(columns={'badl_date'}, inplace=True)
badl.rename(columns={'BADL Adjusted Total':'BADL'}, inplace=True)
badl.rename(columns=lambda x: x.replace('_', ' ').capitalize(), inplace=True)
badl.rename(columns={'Badl':'Baseline BADL', 'Assessment month':'Assessment Month'}, inplace=True)
badl = badl[['Dyad', 'Baseline BADL', 'Food score', 'Eating score', 'Drink score',
       'Drinking score', 'Dressing score', 'Hygiene score', 'Teeth score',
       'Bath shower score', 'Toilet commode score', 'Transfers score',
       'Mobility score', 'Orientation time score', 'Orientation space score',
       'Communication score', 'Telephone score', 'Housework gardening score',
       'Shopping score', 'Finances score', 'Games hobbies score',
       'Transport score', 'Assessment Month']]
badl.rename(columns = {col: f"{col} (Baseline BADL)" for col in badl.loc[:, 'Food score':'Transport score'].columns}, inplace=True)

# Invert BADL scores
cols = badl.loc[:,'Food score (Baseline BADL)':'Transport score (Baseline BADL)'].columns
for col in cols:
    badl[col] = 3 - badl[col]
badl['Baseline BADL'] = 60-badl['Baseline BADL'] 
badl = badl.sort_values(by=['Dyad', 'Assessment Month'])

In [ ]:
mmse_12m['Date'] = pd.to_datetime(mmse_12m['Date']).dt.date
CMB['Date'] = pd.to_datetime(CMB['Date']).dt.date
adas['Date'] = pd.to_datetime(adas['Date']).dt.date

mmse_12m_comb = pd.merge(mmse_12m, CMB, on=['Dyad', 'Date'], how='outer')
mmse_12m_comb = mmse_12m_comb.sort_values(by=['Dyad', 'Date'])

# Fill all MMSE rows with the current comorbidities
columns_to_fill = mmse_12m_comb.loc[:, 'Oncological Comorbidity':'Dermatological Comorbidity'].columns
mmse_12m_comb[columns_to_fill] = mmse_12m_comb.groupby('Dyad')[columns_to_fill].transform(lambda group: group.ffill().fillna(0))
mmse_12m_comb.dropna(inplace=True)

mmse_12_comb = pd.merge(mmse_12m_comb, adas, on=['Dyad','Assessment Month', 'Date'], how='left')
mmse_12_comb = pd.merge(mmse_12_comb, Dem, on='Dyad', how='left')

mmse_12_comb = pd.merge(mmse_12_comb, diagnosis_date, on='Dyad', how='left')

mmse_12_comb['Date'] = pd.to_datetime(mmse_12_comb['Date']).dt.year
mmse_12_comb['Age'] = mmse_12_comb['Date']-mmse_12_comb['YoB']
mmse_12_comb['Diagnosis_date'] = pd.to_datetime(mmse_12_comb['Diagnosis_date']).dt.year
mmse_12_comb['Time since diagnosis'] = mmse_12_comb['Date']-mmse_12_comb['Diagnosis_date']
mmse_12_comb.drop(columns=['Date', 'Assessment Month', 'YoB', 'Diagnosis_date'], inplace=True) 
mmse_12_comb.dropna(inplace=True)

mmse_12_roc_comb = mmse_12_comb.copy()
mmse_12_roc_comb['MMSE_ROC_12'] = mmse_12_roc_comb['MMSE_12m'] - mmse_12_roc_comb['Baseline MMSE']
mmse_12_roc_comb.drop(columns='MMSE_12m', inplace=True)

In [ ]:
mmse_12m_comb_badl = pd.merge(mmse_12m_comb, adas, on=['Dyad','Assessment Month', 'Date'], how='left')
mmse_12m_comb_badl = pd.merge(mmse_12m_comb_badl, Dem, on='Dyad', how='left')

mmse_12m_comb_badl['Date'] = pd.to_datetime(mmse_12m_comb_badl['Date']).dt.year
mmse_12m_comb_badl['Age'] = mmse_12m_comb_badl['Date']-mmse_12m_comb_badl['YoB']

mmse_12m_comb_badl = pd.merge(mmse_12m_comb_badl, diagnosis_date, on='Dyad', how='left')
mmse_12m_comb_badl['Diagnosis_date'] = pd.to_datetime(mmse_12m_comb_badl['Diagnosis_date']).dt.year
mmse_12m_comb_badl['Time since diagnosis'] = mmse_12m_comb_badl['Date']-mmse_12m_comb_badl['Diagnosis_date']
mmse_12m_comb_badl.drop(columns=['Date', 'Diagnosis_date'], inplace=True)

mmse_12m_comb_badl = pd.merge(mmse_12m_comb_badl, badl, on=['Dyad','Assessment Month'], how='left')
mmse_12m_comb_badl.drop(columns=['Assessment Month', 'YoB'], inplace=True)
mmse_12m_comb_badl.dropna(inplace=True)

mmse_12_roc_comb_badl = mmse_12m_comb_badl.copy()
mmse_12_roc_comb_badl['MMSE_ROC_12'] = mmse_12_roc_comb_badl['MMSE_12m'] - mmse_12_roc_comb_badl['Baseline MMSE']
mmse_12_roc_comb_badl.drop(columns='MMSE_12m', inplace=True)

In [ ]:
# Stratifying by dementia type 

DT = pd.read_csv()

DT['Dementia_type'] = DT['Dementia_type'].astype(str)
DT['Dementia_type'] = DT['Dementia_type'].apply(lambda x: 'AD' if x == 'AD, AD' else x)
DT['Dementia_type'] = DT['Dementia_type'].apply(lambda x: 'No dementia' if 'No dementia' in x else x)
DT['Dementia_type'] = DT['Dementia_type'].apply(lambda x: 'Mixed' if ',' in x else x)

AD = DT[(DT['Dementia_type']=='AD')|(DT['Dementia_type']=='MCI')]

mmse = pd.merge(mmse_12_roc_comb, AD, on='Dyad', how='inner')
mmse.drop(columns='Dementia_type', inplace=True)

mmse_badl = pd.merge(mmse_12_roc_comb_badl, AD, on='Dyad', how='inner')
mmse_badl.drop(columns='Dementia_type', inplace=True)

In [ ]:
badl_12m = badl.copy()
badl_12m = badl_12m[(badl_12m['Assessment Month']==0)|(badl_12m['Assessment Month']==12)|(badl_12m['Assessment Month']==24)|
                    (badl_12m['Assessment Month']==36)|(badl_12m['Assessment Month']==48)]
badl_12m['BADL_12m'] = badl_12m.groupby('Dyad')['Baseline BADL'].shift(-1)
badl_12m['AM_12m'] = badl_12m.groupby('Dyad')['Assessment Month'].shift(-1)
badl_12m['AM_dif'] = badl_12m['AM_12m']-badl_12m['Assessment Month']

badl_12m = badl_12m[badl_12m['AM_dif'] == 12.0]
badl_12m.drop(columns=['AM_dif', 'AM_12m'], inplace=True)
badl_12m.dropna(subset=['BADL_12m'], inplace=True)
badl_12m.reset_index(drop=True, inplace=True)

In [ ]:
badl_12m['Date'] = pd.to_datetime(badl_12m['Date']).dt.date
CMB['Date'] = pd.to_datetime(CMB['Date']).dt.date
mmse['Date'] = pd.to_datetime(mmse['Date'])

badl_roc_comb = pd.merge(badl_12m, CMB, on=['Dyad', 'Date'], how='outer')
badl_roc_comb = badl_roc_comb.sort_values(by=['Dyad', 'Date'])

# Fill all MMSE rows with the current comorbidities
columns_to_fill = badl_roc_comb.loc[:, 'Oncological Comorbidity':'Dermatological Comorbidity'].columns  # Get columns between 'Cancer' and 'Skin'
badl_roc_comb[columns_to_fill] = badl_roc_comb.groupby('Dyad')[columns_to_fill].transform(lambda group: group.ffill().fillna(0))

badl_roc_comb = pd.merge(badl_roc_comb, Dem, on='Dyad', how='left')

badl_roc_comb['YoB'] = badl_roc_comb['YoB'].astype(int)
badl_roc_comb['YoB'] = badl_roc_comb['YoB'].apply(lambda year: datetime(year, 1, 1))
badl_roc_comb['Date'] = pd.to_datetime(badl_roc_comb['Date'])
badl_roc_comb['Age'] = badl_roc_comb['Date'] - badl_roc_comb['YoB']
badl_roc_comb['Age'] = round((badl_roc_comb['Age'].dt.days / 365.25), 0)
badl_roc_comb['Age'] = badl_roc_comb['Age'].astype(int)

badl_roc_comb = pd.merge(badl_roc_comb, mmse, on=['Dyad', 'Assessment Month', 'Date'], how='left')
badl_roc_comb = pd.merge(badl_roc_comb, adas, on=['Dyad', 'Assessment Month'], how='left')
badl_roc_comb = pd.merge(badl_roc_comb, diagnosis_date, on='Dyad', how='left')

badl_roc_comb['Date'] = pd.to_datetime(badl_roc_comb['Date']).dt.year
badl_roc_comb['Diagnosis_date'] = pd.to_datetime(badl_roc_comb['Diagnosis_date']).dt.year
badl_roc_comb['Time since diagnosis'] = badl_roc_comb['Date']-badl_roc_comb['Diagnosis_date']
badl_roc_comb.drop(columns=['Date', 'Assessment Month', 'YoB', 'Diagnosis_date'], inplace=True)

badl_roc_comb['BADL ROC'] = badl_roc_comb['BADL_12m'] - badl_roc_comb['Baseline BADL']
badl_roc_comb.drop(columns='BADL_12m', inplace=True)

badl_roc_comb.dropna(inplace=True)

# Stratifying by dementia type 

badl = pd.merge(badl_roc_comb, AD, on='Dyad', how='inner')
badl.drop(columns=['Dementia_type'], inplace=True)

In [ ]:
cmb_cols = ['Oncological Comorbidity',
       'Cardiovascular Comorbidity', 'Digestive Comorbidity',
       'Ear Comorbidity', 'Endocrine Comorbidity', 'Eye Comorbidity',
       'Genitourinary Comorbidity', 'Haematological/Immunological Comorbidity',
       'Infection Comorbidity', 'Musculoskeletal Comorbidity',
       'Neurological Comorbidity', 'Other Comorbidities',
       'Psychiatric Comorbidity', 'Respiratory Comorbidity',
       'Dermatological Comorbidity']
badl_cols = ['Dyad', 'BADL ROC', 'Baseline BADL', 'Food score (Baseline BADL)',
       'Eating score (Baseline BADL)', 'Drink score (Baseline BADL)',
       'Drinking score (Baseline BADL)', 'Dressing score (Baseline BADL)',
       'Hygiene score (Baseline BADL)', 'Teeth score (Baseline BADL)',
       'Bath shower score (Baseline BADL)',
       'Toilet commode score (Baseline BADL)',
       'Transfers score (Baseline BADL)', 'Mobility score (Baseline BADL)',
       'Orientation time score (Baseline BADL)',
       'Orientation space score (Baseline BADL)',
       'Communication score (Baseline BADL)',
       'Telephone score (Baseline BADL)',
       'Housework gardening score (Baseline BADL)',
       'Shopping score (Baseline BADL)', 'Finances score (Baseline BADL)',
       'Games hobbies score (Baseline BADL)',
       'Transport score (Baseline BADL)', 'Gender', 'Age', 'Year (Baseline MMSE)',
       'Season (Baseline MMSE)', 'Month (Baseline MMSE)',
       'Date (Baseline MMSE)', 'Day (Baseline MMSE)', 'Contry (Baseline MMSE)',
       'County (Baseline MMSE)', 'City (Baseline MMSE)',
       'Building (Baseline MMSE)', 'Floor (Baseline MMSE)',
       'Registration (Baseline MMSE)', 'World (Baseline MMSE)',
       'Recall (Baseline MMSE)', 'Naming (Baseline MMSE)',
       'Repeat (Baseline MMSE)', 'Closeyoureyes (Baseline MMSE)',
       'Writesentence (Baseline MMSE)', 'Copypentagons (Baseline MMSE)',
       'Commands (Baseline MMSE)', 'Baseline MMSE', 'Baseline ADAS-Cog',
       'Spoken language (Baseline ADAS)',
       'Comprehension language (Baseline ADAS)',
       'Word finding (Baseline ADAS)',
       'Remembering instructions (Baseline ADAS)',
       'Word recall (Baseline ADAS)', 'Naming (Baseline ADAS)',
       'Commands (Baseline ADAS)', 'Constructional praxis (Baseline ADAS)',
       'Delayed recall (Baseline ADAS)', 'Ideational praxis (Baseline ADAS)',
       'Orientation (Baseline ADAS)', 'Word recognition (Baseline ADAS)',
       'Maze (Baseline ADAS)', 'Number cancellation (Baseline ADAS)', 'Time since diagnosis'] # ,'Count'
mmse_cols = ['Dyad', 'MMSE_ROC_12', 'Year (Baseline MMSE)', 'Season (Baseline MMSE)',
       'Month (Baseline MMSE)', 'Date (Baseline MMSE)', 'Day (Baseline MMSE)',
       'Contry (Baseline MMSE)', 'County (Baseline MMSE)',
       'City (Baseline MMSE)', 'Building (Baseline MMSE)',
       'Floor (Baseline MMSE)', 'Registration (Baseline MMSE)',
       'World (Baseline MMSE)', 'Recall (Baseline MMSE)',
       'Naming (Baseline MMSE)', 'Repeat (Baseline MMSE)',
       'Closeyoureyes (Baseline MMSE)', 'Writesentence (Baseline MMSE)',
       'Copypentagons (Baseline MMSE)', 'Commands (Baseline MMSE)',
       'Baseline MMSE', 'Baseline ADAS-Cog', 'Spoken language (Baseline ADAS)',
       'Comprehension language (Baseline ADAS)',
       'Word finding (Baseline ADAS)',
       'Remembering instructions (Baseline ADAS)',
       'Word recall (Baseline ADAS)', 'Naming (Baseline ADAS)',
       'Commands (Baseline ADAS)', 'Constructional praxis (Baseline ADAS)',
       'Delayed recall (Baseline ADAS)', 'Ideational praxis (Baseline ADAS)',
       'Orientation (Baseline ADAS)', 'Word recognition (Baseline ADAS)',
       'Maze (Baseline ADAS)', 'Number cancellation (Baseline ADAS)', 'Gender',
       'Age', 'Time since diagnosis']
mmse_badl_cols = ['Dyad', 'MMSE_ROC_12', 'Year (Baseline MMSE)', 'Season (Baseline MMSE)',
       'Month (Baseline MMSE)', 'Date (Baseline MMSE)', 'Day (Baseline MMSE)',
       'Contry (Baseline MMSE)', 'County (Baseline MMSE)',
       'City (Baseline MMSE)', 'Building (Baseline MMSE)',
       'Floor (Baseline MMSE)', 'Registration (Baseline MMSE)',
       'World (Baseline MMSE)', 'Recall (Baseline MMSE)',
       'Naming (Baseline MMSE)', 'Repeat (Baseline MMSE)',
       'Closeyoureyes (Baseline MMSE)', 'Writesentence (Baseline MMSE)',
       'Copypentagons (Baseline MMSE)', 'Commands (Baseline MMSE)',
       'Baseline MMSE', 'Baseline ADAS-Cog', 'Spoken language (Baseline ADAS)',
       'Comprehension language (Baseline ADAS)',
       'Word finding (Baseline ADAS)',
       'Remembering instructions (Baseline ADAS)',
       'Word recall (Baseline ADAS)', 'Naming (Baseline ADAS)',
       'Commands (Baseline ADAS)', 'Constructional praxis (Baseline ADAS)',
       'Delayed recall (Baseline ADAS)', 'Ideational praxis (Baseline ADAS)',
       'Orientation (Baseline ADAS)', 'Word recognition (Baseline ADAS)',
       'Maze (Baseline ADAS)', 'Number cancellation (Baseline ADAS)', 'Gender',
       'Age', 'Time since diagnosis', 'Baseline BADL', 'Food score (Baseline BADL)',
       'Eating score (Baseline BADL)', 'Drink score (Baseline BADL)',
       'Drinking score (Baseline BADL)', 'Dressing score (Baseline BADL)',
       'Hygiene score (Baseline BADL)', 'Teeth score (Baseline BADL)',
       'Bath shower score (Baseline BADL)',
       'Toilet commode score (Baseline BADL)',
       'Transfers score (Baseline BADL)', 'Mobility score (Baseline BADL)',
       'Orientation time score (Baseline BADL)',
       'Orientation space score (Baseline BADL)',
       'Communication score (Baseline BADL)',
       'Telephone score (Baseline BADL)',
       'Housework gardening score (Baseline BADL)',
       'Shopping score (Baseline BADL)', 'Finances score (Baseline BADL)',
       'Games hobbies score (Baseline BADL)',
       'Transport score (Baseline BADL)']
mmse_only_cols = ['Year (Baseline MMSE)', 'Season (Baseline MMSE)',
       'Month (Baseline MMSE)', 'Date (Baseline MMSE)', 'Day (Baseline MMSE)',
       'Contry (Baseline MMSE)', 'County (Baseline MMSE)',
       'City (Baseline MMSE)', 'Building (Baseline MMSE)',
       'Floor (Baseline MMSE)', 'Registration (Baseline MMSE)',
       'World (Baseline MMSE)', 'Recall (Baseline MMSE)',
       'Naming (Baseline MMSE)', 'Repeat (Baseline MMSE)',
       'Closeyoureyes (Baseline MMSE)', 'Writesentence (Baseline MMSE)',
       'Copypentagons (Baseline MMSE)', 'Commands (Baseline MMSE)',
       'Baseline MMSE']
adas_only_cols = ['Baseline ADAS-Cog', 'Spoken language (Baseline ADAS)',
       'Comprehension language (Baseline ADAS)',
       'Word finding (Baseline ADAS)',
       'Remembering instructions (Baseline ADAS)',
       'Word recall (Baseline ADAS)', 'Naming (Baseline ADAS)',
       'Commands (Baseline ADAS)', 'Constructional praxis (Baseline ADAS)',
       'Delayed recall (Baseline ADAS)', 'Ideational praxis (Baseline ADAS)',
       'Orientation (Baseline ADAS)', 'Word recognition (Baseline ADAS)',
       'Maze (Baseline ADAS)', 'Number cancellation (Baseline ADAS)']

In [ ]:
mmse_cmb = mmse[mmse_cols+cmb_cols]
badl_cmb = badl[badl_cols+cmb_cols]

mmse_cmb.dropna(inplace=True)
badl_cmb.dropna(inplace=True)

mmse_badl_cmb = mmse_badl[mmse_badl_cols+cmb_cols]
mmse_badl = mmse_badl[mmse_badl_cols]

mmse = mmse_cmb.drop(columns=cmb_cols)
badl = badl_cmb.drop(columns=cmb_cols)

badl_nommse = badl.drop(columns=mmse_only_cols)
badl_noadas = badl.drop(columns=adas_only_cols)
badl_nocog = badl_noadas.drop(columns=mmse_only_cols)

badl_dfs = [badl, badl_cmb, badl_nommse, badl_noadas, badl_nocog]
mmse_dfs = [mmse,mmse_cmb,mmse_badl,mmse_badl_cmb]

for df in badl_dfs:
    df = df.drop_duplicates(subset=['Dyad','Baseline BADL', 'Age'], keep='last')
    df['BADL ROC'] = winsorize(df['BADL ROC'], limits=[0.05, 0.05])

for df in mmse_dfs:
    df = df.drop_duplicates(subset=['Dyad','Baseline MMSE', 'Age'], keep='last')
    df['MMSE_ROC_12'] = winsorize(df['MMSE_ROC_12'], limits=[0.05, 0.05])